In [1]:
#Import Dependencies 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

In [2]:
state_abb = "Resources/state-abbrevs.csv"
state_area = "Resources/state-areas.csv"

In [3]:
state_abb_read = pd.read_csv(state_abb)
state_abb_df = pd.DataFrame(state_abb_read)
state_area_read = pd.read_csv(state_area)
state_area_df = pd.DataFrame(state_area_read)
state_info_df = pd.merge(state_abb_df, state_area_df)

In [6]:
covid_file= "Resources/all-states-history-1219.csv"
covid_data=pd.read_csv(covid_file)
covid_df = pd.DataFrame(covid_data) 
covid_df.rename(columns = {"state" : "abbreviation"}, inplace = True)
covid_df.head()

,date,abbreviation,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,12/18/20,AK,A,183.0,183.0,0,NaN,928.0,928.0,142.0,...,1184922.0,9199,NaN,NaN,NaN,NaN,NaN,0,1184922.0,9199
1,12/18/20,AL,A,4296.0,3772.0,42,524.0,30224.0,30224.0,2447.0,...,1764750.0,12037,NaN,NaN,80872.0,NaN,1764750.0,12037,NaN,0
2,12/18/20,AR,A+,3139.0,2776.0,27,363.0,10430.0,10430.0,1073.0,...,1913448.0,13674,NaN,21856.0,NaN,198712.0,NaN,0,1913448.0,13674
3,12/18/20,AS,D,0.0,NaN,0,NaN,NaN,NaN,NaN,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
4,12/18/20,AZ,A+,7819.0,7142.0,142,677.0,32293.0,32293.0,3931.0,...,4514548.0,51882,381780.0,NaN,NaN,NaN,2605792.0,22398,4514548.0,51882


In [36]:
#convert date column to column
covid_df['date'] = pd.to_datetime(covid_df['date'])
covid_df

,date,abbreviation,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2020-12-18,AK,A,183.0,183.0,0,NaN,928.0,928.0,142.0,...,1184922.0,9199,NaN,NaN,NaN,NaN,NaN,0,1184922.0,9199
1,2020-12-18,AL,A,4296.0,3772.0,42,524.0,30224.0,30224.0,2447.0,...,1764750.0,12037,NaN,NaN,80872.0,NaN,1764750.0,12037,NaN,0
2,2020-12-18,AR,A+,3139.0,2776.0,27,363.0,10430.0,10430.0,1073.0,...,1913448.0,13674,NaN,21856.0,NaN,198712.0,NaN,0,1913448.0,13674
3,2020-12-18,AS,D,0.0,NaN,0,NaN,NaN,NaN,NaN,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
4,2020-12-18,AZ,A+,7819.0,7142.0,142,677.0,32293.0,32293.0,3931.0,...,4514548.0,51882,381780.0,NaN,NaN,NaN,2605792.0,22398,4514548.0,51882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16310,2020-01-17,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
16311,2020-01-16,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
16312,2020-01-15,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
16313,2020-01-14,WA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [50]:
#merge the data files 
covid_state = pd.merge(state_info_df, covid_df, on = "abbreviation")
covid_state

,state,abbreviation,area (sq. mi),date,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,Alabama,AL,52423,2020-12-18,A,4296.0,3772.0,42,524.0,30224.0,...,1764750.0,12037,NaN,NaN,80872.0,NaN,1764750.0,12037,NaN,0
1,Alabama,AL,52423,2020-12-17,A,4254.0,3745.0,56,509.0,29559.0,...,1752713.0,11585,NaN,NaN,80026.0,NaN,1752713.0,11585,NaN,0
2,Alabama,AL,52423,2020-12-16,A,4198.0,3704.0,74,494.0,29559.0,...,1741128.0,8943,NaN,NaN,79047.0,NaN,1741128.0,8943,NaN,0
3,Alabama,AL,52423,2020-12-15,A,4124.0,3642.0,22,482.0,29259.0,...,1732185.0,8516,NaN,NaN,78493.0,NaN,1732185.0,8516,NaN,0
4,Alabama,AL,52423,2020-12-14,A,4102.0,3624.0,0,478.0,28913.0,...,1723669.0,27230,NaN,NaN,78262.0,NaN,1723669.0,27230,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14920,Wyoming,WY,97818,2020-03-05,NaN,NaN,NaN,0,NaN,NaN,...,8.0,4,NaN,NaN,NaN,NaN,NaN,0,8.0,4
14921,Wyoming,WY,97818,2020-03-04,NaN,NaN,NaN,0,NaN,NaN,...,4.0,3,NaN,NaN,NaN,NaN,NaN,0,4.0,3
14922,Wyoming,WY,97818,2020-03-03,NaN,NaN,NaN,0,NaN,NaN,...,1.0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0
14923,Wyoming,WY,97818,2020-03-02,NaN,NaN,NaN,0,NaN,NaN,...,1.0,0,NaN,NaN,NaN,NaN,NaN,0,1.0,0


In [51]:
# covid_state.to_csv("COVID_State.csv", index = False)

covid_state = covid_state[["state", "abbreviation", "area (sq. mi)", "date", "dataQualityGrade", "death", "hospitalized", "positive", "negative", "recovered", "totalTestResults"]]
covid_state



,state,abbreviation,area (sq. mi),date,dataQualityGrade,death,hospitalized,positive,negative,recovered,totalTestResults
0,Alabama,AL,52423,2020-12-18,A,4296.0,30224.0,315683.0,1507204.0,183625.0,1764750.0
1,Alabama,AL,52423,2020-12-17,A,4254.0,29559.0,310335.0,1499349.0,183625.0,1752713.0
2,Alabama,AL,52423,2020-12-16,A,4198.0,29559.0,305640.0,1491279.0,183625.0,1741128.0
3,Alabama,AL,52423,2020-12-15,A,4124.0,29259.0,301533.0,1485182.0,174805.0,1732185.0
4,Alabama,AL,52423,2020-12-14,A,4102.0,28913.0,297895.0,1478907.0,174805.0,1723669.0
...,...,...,...,...,...,...,...,...,...,...,...
14920,Wyoming,WY,97818,2020-03-05,NaN,NaN,NaN,NaN,NaN,NaN,8.0
14921,Wyoming,WY,97818,2020-03-04,NaN,NaN,NaN,NaN,NaN,NaN,4.0
14922,Wyoming,WY,97818,2020-03-03,NaN,NaN,NaN,NaN,NaN,NaN,1.0
14923,Wyoming,WY,97818,2020-03-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [44]:
#identify most recent date in the dataframe
covid_state = covid_state.sort_values("date", ascending=False)
covid_state

,state,abbreviation,area (sq. mi),date,dataQualityGrade,death,hospitalized,positive,negative,recovered,totalTestResults
0,Alabama,AL,52423,2020-12-18,A,4296.0,30224.0,315683.0,1507204.0,183625.0,1764750.0
1733,Connecticut,CT,5544,2020-12-18,C,5581.0,12257.0,162782.0,3782866.0,9800.0,3945648.0
2922,Georgia,GA,59441,2020-12-18,A+,10332.0,39064.0,500265.0,4403781.0,NaN,4904046.0
11388,Rhode Island,RI,1545,2020-12-18,A+,1625.0,5822.0,77812.0,519811.0,4761.0,1817360.0
575,Arizona,AZ,114006,2020-12-18,A+,7819.0,32293.0,442671.0,2181193.0,65350.0,4514548.0
...,...,...,...,...,...,...,...,...,...,...,...
14048,Washington,WA,71303,2020-01-17,NaN,NaN,NaN,0.0,NaN,NaN,NaN
14049,Washington,WA,71303,2020-01-16,NaN,NaN,NaN,0.0,NaN,NaN,NaN
14050,Washington,WA,71303,2020-01-15,NaN,NaN,NaN,0.0,NaN,NaN,NaN
14051,Washington,WA,71303,2020-01-14,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [49]:
#isolate the data from the most recent report
old_covid = covid_state[(covid_state['date'] != (covid_state['date'].max()))].index
old_covid

#create a dataframe of only the most recent data
recent_df =covid_state.drop(old_covid, inplace=False)
recent_df

,state,abbreviation,area (sq. mi),date,dataQualityGrade,death,hospitalized,positive,negative,recovered,totalTestResults
0,Alabama,AL,52423,2020-12-18,A,4296.0,30224.0,315683.0,1507204.0,183625.0,1764750.0
1733,Connecticut,CT,5544,2020-12-18,C,5581.0,12257.0,162782.0,3782866.0,9800.0,3945648.0
2922,Georgia,GA,59441,2020-12-18,A+,10332.0,39064.0,500265.0,4403781.0,NaN,4904046.0
11388,Rhode Island,RI,1545,2020-12-18,A+,1625.0,5822.0,77812.0,519811.0,4761.0,1817360.0
575,Arizona,AZ,114006,2020-12-18,A+,7819.0,32293.0,442671.0,2181193.0,65350.0,4514548.0
8171,North Dakota,ND,70704,2020-12-18,B,1231.0,3349.0,89557.0,280865.0,85271.0,1193656.0
10526,Mississippi,MS,48434,2020-12-18,A+,4354.0,7901.0,190411.0,1079660.0,148466.0,1270071.0
11100,Pennsylvania,PA,46058,2020-12-18,A+,13608.0,NaN,538655.0,3118874.0,323193.0,3657529.0
5526,Maine,ME,35387,2020-12-18,A+,281.0,965.0,18337.0,1007303.0,10744.0,1025640.0
12258,Tennessee,TN,42146,2020-12-18,A+,5845.0,13537.0,493230.0,4703628.0,418724.0,5196858.0


In [52]:
#add columns to house new data
recent_df["Median Income"]=""
recent_df["Population Density"]=""

recent_df

,state,abbreviation,area (sq. mi),date,dataQualityGrade,death,hospitalized,positive,negative,recovered,totalTestResults,Median Income,Population Density
0,Alabama,AL,52423,2020-12-18,A,4296.0,30224.0,315683.0,1507204.0,183625.0,1764750.0,,
1733,Connecticut,CT,5544,2020-12-18,C,5581.0,12257.0,162782.0,3782866.0,9800.0,3945648.0,,
2922,Georgia,GA,59441,2020-12-18,A+,10332.0,39064.0,500265.0,4403781.0,NaN,4904046.0,,
11388,Rhode Island,RI,1545,2020-12-18,A+,1625.0,5822.0,77812.0,519811.0,4761.0,1817360.0,,
575,Arizona,AZ,114006,2020-12-18,A+,7819.0,32293.0,442671.0,2181193.0,65350.0,4514548.0,,
8171,North Dakota,ND,70704,2020-12-18,B,1231.0,3349.0,89557.0,280865.0,85271.0,1193656.0,,
10526,Mississippi,MS,48434,2020-12-18,A+,4354.0,7901.0,190411.0,1079660.0,148466.0,1270071.0,,
11100,Pennsylvania,PA,46058,2020-12-18,A+,13608.0,NaN,538655.0,3118874.0,323193.0,3657529.0,,
5526,Maine,ME,35387,2020-12-18,A+,281.0,965.0,18337.0,1007303.0,10744.0,1025640.0,,
12258,Tennessee,TN,42146,2020-12-18,A+,5845.0,13537.0,493230.0,4703628.0,418724.0,5196858.0,,


In [ ]:
#pull census populations from API 

In [ ]:
#add a column to the covid_state data frame that calculates population density 


In [ ]:
# covid_state.to_csv("COVID_State.csv", index = False)